For the Sierra Leone, we are also interested in the approximate walking time to each class of destinations. Hence, we will take the final_G and final_G_flood graphs, and add an additional parameter for each edge - time_walk. 

Our base walk speed will be 4.5km/h for each road type. We will add an impedance for flood depth once again for nodes which are flooded. 

In [1]:
import os, sys
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point, MultiPoint
from shapely.wkt import loads
from scipy import spatial
from functools import partial
import pyproj
from shapely.ops import transform
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
import GOSTnet as gn
import rasterio
import numpy as np

networkx version: 2.3 
osmnx version: 0.9 


This is a simple function for adding a fixed speed walk time to each edge in the graph. It adds a new property to the data dictionary called 'walk_time', which converts the edge lengths to times, measured in seconds. 

In [2]:
def addWalking(G, walk_speed):
    G2 = G.copy()
    for u, v, data in G2.edges(data = True):
        data['walk_time'] = ((data['length'] / (walk_speed * 1000)) * 3600)
    return G2

Basic settings. Here, we set our assumed walkspeed

In [3]:
basepth = r'C:\Users\charl\Documents\GOST\SierraLeone\RoadNet'

walk_speed = 4.5

Read in graphs from the previous step

In [4]:
final_G = nx.read_gpickle(os.path.join(basepth, 'final_G.pickle'))

final_G_flood = nx.read_gpickle(os.path.join(basepth, 'final_G_flood.pickle'))

Apply our walking function to both the flooded and non-flooded graph

In [5]:
final_G_walking = addWalking(final_G, walk_speed)

final_G_flood_walking = addWalking(final_G_flood, walk_speed)

Here, we take our network disruption function from the last notebook (Step 4.a) and apply it to walk times instead of drive times.

In [6]:
def disrupt_network(G, attr, thresh, fail_value = 99999999):
    
    #### Function for disrupting a graph ####
    # REQUIRED: G - a graph containing one or more nodes and one or more edges
    #           property - the element in the data dictionary for the edges to test
    #           thresh - values of data[property] above this value are disrupted
    #           fail_value - the data['time'] property is set to this value to simulate the removal of the edge
    # RETURNS:  a modified graph with the edited 'time' attribute
    # -------------------------------------------------------------------------#

    G_copy = G.copy()     # copy the input graph

    broken_nodes, disrupted_nodes = [], []    # generate some empty list objects
    
    disruption_dict = {}   # generate an empty dictionary

    for u, data in G_copy.nodes(data = True):   # Iterate through all nodes, one at a time

        if data[attr] > thresh:    # if larger than the threshold...

            broken_nodes.append(u)    # record this as a broken node
            
            disruption_dict[u] = data[attr]  # add node ID and corresponding flood depth to a dictionary
            
        elif 0 < data[attr] < thresh:    # if flood depth is bigger than 0 - but smaller than thresh:
            
            disrupted_nodes.append(u)   # record this as a disrupted node
            
            disruption_dict[u] = data[attr]   # add node ID and corresponding flood depth to a dictionary
        
        else: # If this node is untouched by the flood....
            
            disruption_dict[u] = 0    # ....set its dictionary value to 0
            
    i = 0
    
    for u, v, data in G_copy.edges(data = True):     # starting a second iteration, this time of edges:

        if u in broken_nodes or v in broken_nodes:    # if start or end node is a broken node

            data['walk_time'] = fail_value    # prevent travel along this edge - set travel time to high value
            i+=1
        
        elif u in disrupted_nodes or v in disrupted_nodes: # if start or end node is a disrupted node
            
            depth = max(disruption_dict[u], disruption_dict[v]) # flood depth is max of depth of flood at u and v
            
            orig_time = data['walk_time'] 
            
            data['walk_time'] = orig_time * (1/ ((thresh - depth) / thresh))  # linearly increase travel times 

    print('edges disrupted: %s' % i)
    
    return G_copy

Apply the disruption function we developed for flooding against our walking times instead of driving times. This assumes that both driving and walking are equally disrupted in the event of a flood

In [7]:
final_G_flood_walking = disrupt_network(final_G_flood_walking, 'flood_depth', 0.5)

edges disrupted: 7208


Save files down

In [9]:
gn.save(final_G_walking, 'final_G_walking', basepth, nodes = False,  edges = False)

gn.save(final_G_flood_walking, 'final_G_flood_walking', basepth, nodes = False,  edges = False)